In [1]:
import tensorflow as tf
sess = tf.compat.v1.Session()

tf.compat.v1.keras.backend.set_session(sess)

In [2]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import zeros
from numpy import ones
from numpy.random import randint

import re

import json

import pandas as pd
from pandas import json_normalize

from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU

from matplotlib import pyplot

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [3]:
def loadData(path):
    with open(path,'r') as handle:
        text_data = handle.read()
        json_data = json.loads(text_data)
        df = pd.DataFrame(json_data)
    return df


#Adding frame numbers for time series plots
def LabelFrame(data):   
    df = pd.DataFrame(data)
    for i in range(len(df.ObjectList.index)):
        for j in df['ObjectList'][i]:
            j['Frame'] = i
            
    df.to_json('temp.json',orient = 'records')
    with open('temp.json') as file:
        data2 = file.read()
        jasondata = json.loads(data2)
    return jasondata


def openfile(path):
    with open(path, 'r') as handle:
        text_data = handle.read()
        text_data = '[' + re.sub(r'\}\s\{','},{', text_data) + ']'
        json_data = json.loads(text_data)
        json_data = LabelFrame(json_data)
    return json_data


#Seperating pk array values into columns
def mergedframe1(df_1):
        frames = [df_1]
        df_merged = pd.concat(frames,ignore_index = True)
        df_classify = pd.DataFrame()
        df_classify[['ObjType','pkArray','dpplIdx']] = df_merged[['ObjType','pkArray','dpplIdx']]
        
        df_classify['pkArray'] = df_classify['pkArray'].apply(lambda pkarray:np.ravel(np.transpose(pkarray)))
        
        all_list = []

        for i in range(35):
            for j in range(len(df_classify.index)):
                all_list.append(df_classify.pkArray[j][i])
        chunk = [all_list[i:i+len(df_classify.index)] for i in range(0, len(all_list), len(df_classify.index))]
        df_classify1 = pd.DataFrame(chunk)
        dfclassify=df_classify1.transpose()
        dfclassify['ObjType'] = df_classify['ObjType']
        dfclassify['rangeIdx'] = df_merged['rangeIdx']
        dfclassify['dpplIdx'] = df_merged['dpplIdx']
        dfclassify['pkArray'] = df_merged['pkArray']
        dfclassify['Range'] = df_merged['Range']
        dfclassify['Coordinates.X'] = df_merged['Coordinates.X']
        dfclassify['Coordinates.Y'] = df_merged['Coordinates.Y']
        dfclassify["pkValue1.Real"] = df_merged["pkValue1.Real"]
        dfclassify["pkValue1.Imaginary"] = df_merged['pkValue1.Imaginary']
        dfclassify["pkValue2.Real"] = df_merged["pkValue2.Real"]
        dfclassify["pkValue2.Imaginary"] = df_merged['pkValue2.Imaginary']
        dfclassify["pkValue3.Real"] = df_merged["pkValue3.Real"]
        dfclassify["pkValue3.Imaginary"] = df_merged['pkValue3.Imaginary']
        dfclassify["pkValue4.Real"] = df_merged["pkValue4.Real"]
        dfclassify["pkValue4.Imaginary"] = df_merged['pkValue4.Imaginary']
        
        dfclassify['Frame'] = df_merged['Frame']
        dfclassify['Range'] = df_merged['Range']
        return dfclassify


# Load all files from json to a dataframe after 
def startdf(json_data,Label):
    df_1= pd.DataFrame()
    df1 = pd.DataFrame((json_data))
    for j in range(len(df1.index)):
        df2 = json_normalize(df1['ObjectList'][j])
        df2['ObjType'] = df2['ObjType'].map(lambda ObjType: Label)        
        #df2 = df2[df2.Range <= UIdx]
        #df2 = df2[df2.Range >= LIdx]
        #df2 = df2[df2.dpplIdx == 1]   
        df_1 = pd.concat([df2,df_1],ignore_index =True)
    df_1 = df_1.sort_values(by=['Frame'],ascending = True)
    df_1 = df_1.reset_index(drop=True)
    df_1['pkArray'] = df_1['pkArray'].map(lambda x:np.transpose(x))
    df_final = mergedframe1(df_1)
    return df_final


def labelData(df, x1, x2, y1, y2, label):
    df['ObjType'] = df['ObjType'].map(lambda x : 0)

    for index, row in df.iterrows():
        if ((row['Coordinates.X'] > x1) & (row['Coordinates.X'] < x2) 
        & (row['Coordinates.Y'] > y1) & (row['Coordinates.Y'] < y2)):
            df.at[index, 'ObjType'] = label
        else:
            df.at[index, 'ObjType'] = 0
    return df


# Shuffle two arrays in unison
def unison_shuffled_copies(array1, array2):
    assert len(array1) == len(array2)
    p = np.random.permutation(len(array1))
    return array1[p], array2[p]


def padHumanRows(df):
  for ind, row in df.iterrows():
    if ((ind % 10) == 0):
      count = 0
      while ((count < 10) and (df.iloc[ind + count]['Frame'] == int(ind/10))):
        count = count + 1
        if ((ind + count) == len(df.index)):
          break
      while (count < 10):
        row = pd.DataFrame({"0": 0, "1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "18": 0, "19": 0, "20": 0, "21": 0, "22": 0, "23": 0, "24": 0, "25": 0, "26": 0, "27": 0, "28": 0, "29": 0, "30": 0, "31": 0, "32": 0, "33": 0, "34": 0, "rangeIdx":0, "dpplIdx":0, "Frame": ind/10, "ObjType":0}, index=[ind+count-1+0.5])
        df = df.append(row, ignore_index=False)
        df = df.sort_index().reset_index(drop=True)
        df = df.reindex(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "rangeIdx", "dpplIdx", "Frame", "ObjType"], axis=1)
        count = count + 1
  return df

def padCarRows(df):
  for ind, row in df.iterrows():
    if ((ind % 10) == 0):
      count = 0
      while ((count < 10) and (df.iloc[ind + count]['Frame'] == int(ind/10))):
        count = count + 1
        if ((ind + count) == len(df.index)):
          break
      while (count < 10):
        row = pd.DataFrame({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, "rangeIdx":0, "dpplIdx":0, "Frame": ind/10, "ObjType":0}, index=[ind+count-1+0.5])
        df = df.append(row, ignore_index=False)
        df = df.sort_index().reset_index(drop=True)
        df = df.reindex([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, "rangeIdx", "dpplIdx", "Frame", "ObjType"], axis=1)
        count = count + 1
  return df


def padRows(df, obj):
  if (obj == "car"):
    while (df.shape[0] != ((df['Frame'].max() + 1)*10)):
      df = padCarRows(df)
  else:
    while (df.shape[0] != ((df['Frame'].max() + 1)*10)):
      df = padHumanRows(df)  
  return df


def truncateColumns(df):
  df = df.drop(['Range', 'pkArray', 'Coordinates.X', 'Coordinates.Y', 'pkValue1.Real', 
           'pkValue1.Imaginary', 'pkValue2.Real', 'pkValue2.Imaginary', 
           'pkValue3.Real', 'pkValue3.Imaginary', 'pkValue4.Real', 
           'pkValue4.Imaginary', 'ObjType'], axis=1)
  return df


def dfTo2DArray(df):
  df = df[df.columns[0:38]].values
  return df

#to correct the dppl idx
def correctDpplIdx(x):
    if x <=31:
        return x
    if x>31:
        return x-64


# frameTo128x64() function converts a 2D input array
# where rows == objects in json file and cols == 0,1,..34 peak array elements, rangeIdx, dopplIdx, Frame.
# The 2D array is converted to 3D array of size no. of frames * 128 * 64

def frameTo128x64(arr):
    count = 0
    new3DArray = np.empty((1, 128, 64))
    newLabelArr = np.empty((1, 128, 64))
    arr128x64 = np.zeros((128, 64))
    labelArr = np.zeros((128, 64))
    for row in range(0, np.shape(arr)[0], 1):
        if count<=10:
            rangeIndex = int(arr[row][35])
            dopplerIndex = int(correctDpplIdx(arr[row][36]))
            label = arr[row][37]
            r = 0
            
            for rangeID in range(rangeIndex-3, rangeIndex+4, 1):
                d = 0
                for dopplID in range(dopplerIndex-2, dopplerIndex+3, 1):
                    if ((0 <= rangeID <= 127) and (-32 <= dopplID <= 31)):
                        if(arr128x64[rangeID][dopplID+32] < arr[row][(d * 7) + r]):
                            arr128x64[rangeID][dopplID+32] = arr[row][(d * 7) + r]
                        if(labelArr[rangeID][dopplID+32] == 0):
                            labelArr[rangeID][dopplID+32] = label
                    d = d + 1
                r = r + 1
        count= count + 1
        if count == 10:
            count = 0
            arr128x64 = np.reshape(arr128x64, (1, 128, 64))
            labelArr = np.reshape(labelArr, (1, 128, 64))
            new3DArray = np.append(new3DArray, arr128x64, axis=0)
            newLabelArr = np.append(newLabelArr, labelArr, axis=0)
            arr128x64 = np.zeros((128, 64))
            labelArr = np.zeros((128, 64))
    return new3DArray[1:, :, :], newLabelArr[1:, :, :]

In [17]:
humanDay1_2pm = loadData("/content/RadarDataset/human/human_2pm.json")
humanDay1_5pm = loadData("/content/RadarDataset/human/human_5pm.json")
#/content/RadarDataset/human/human_2pm.json
humanDay1_9pm = loadData("/content/RadarDataset/human/human_9pm.json")
humanDay1_3pm = loadData("/content/RadarDataset/human/human_3pm.json")
humanDay1_6pm = loadData("/content/RadarDataset/human/human_6pm.json")

In [18]:
carVday2_5 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_5_28_02_2020 12_38_17.json"), 2)
carVday1_6 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_6_27_01_2020 12_42_51.json"), 2)
carVday1_10 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_10_27_01_2020 12_46_59.json"), 2)
carVday2_11 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_11_28_02_2020 13_00_06.json"), 2)
carVday2_15 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_15_28_02_2020 13_23_56.json"), 2)
carVday1_16 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_16_27_01_2020 12_53_52.json"), 2)
carVday2_20 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_20_28_02_2020 15_38_52.json"), 2)
carVday1_19 = startdf(openfile("/content/RadarDataset/carV/Car Vertical_19_27_01_2020 12_57_35.json"), 2)

In [19]:
carHday1_5 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_5_27_01_2020 13_05_27.json"), 3)
carHday1_7 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_7_27_01_2020 13_07_36.json"), 3)
carHday1_11 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_11_27_01_2020 13_40_38.json"), 3)
carHday1_17 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_17_27_01_2020 13_17_50.json"), 3)
carHday1_20 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_20_27_01_2020 13_21_11.json"), 3)
carHday2_2 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_2_12_03_2020 14_50_24.json"), 3)
carHday2_4 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_4_12_03_2020 14_53_38.json"), 3)
carHday2_6 = startdf(openfile("/content/RadarDataset/carH/Car Horizontal_6_12_03_2020 15_00_25.json"), 3)

In [20]:
arrHumanDay1_2pm, labelHumanDay1_2pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_2pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_5pm, labelHumanDay1_5pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_5pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_9pm, labelHumanDay1_9pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_9pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_3pm, labelHumanDay1_3pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_3pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))
arrHumanDay1_6pm, labelHumanDay1_6pm = frameTo128x64(dfTo2DArray(padRows(labelData(humanDay1_6pm, -0.5, 0.5, 0.1, 7.9, 1), "human")))

In [21]:
arrCarVday1_6, labelCarVday1_6 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_6, -1.5, 1.5, 5.5, 7.5, 2), "car")))
arrCarVday1_10, labelCarVday1_10 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_10, -1.5, 1.5, 9.5, 11.5, 2), "car")))
arrCarVday1_16, labelCarVday1_16 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_16, -1.5, 1.5, 15.5, 17.5, 2), "car")))
arrCarVday1_19, labelCarVday1_19 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_19, -1.5, 1.5, 18.5, 20.5, 2), "car")))
arrCarVday2_5, labelCarVday2_5 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_5, -1.5, 1.5, 4.5, 6.5, 2), "car")))
arrCarVday2_15, labelCarVday2_15 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_15, -1.5, 1.5, 14.5, 16.5, 2), "car")))
arrCarVday2_11, labelCarVday2_11 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_11, -1.5, 1.5, 10.5, 12.5, 2), "car")))
arrCarVday2_20, labelCarVday2_20 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_20, -1.5, 1.5, 19.5, 21.5, 2), "car")))

In [22]:
arrCarHday1_5, labelCarHday1_5 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_5, -3, 3, 4.5, 7, 3), "car")))
arrCarHday1_7, labelCarHday1_7 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_7, -3, 3, 6.5, 9, 3), "car")))
arrCarHday1_11, labelCarHday1_11 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_11, -3, 3, 10.5, 13, 3), "car")))
arrCarHday1_17, labelCarHday1_17 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_17, -3, 3, 16.5, 19, 3), "car")))
arrCarHday1_20, labelCarHday1_20 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_20, -3, 3, 19.5, 22, 3), "car")))
arrCarHday2_2, labelCarHday2_2 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_2, -3, 3, 1.5, 4, 3), "car")))
arrCarHday2_4, labelCarHday2_4 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_4, -3, 3, 3.5, 6, 3), "car")))
arrCarHday2_6, labelCarHday2_6 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_6, -3, 3, 5.5, 8, 3), "car")))

In [23]:
print("Human Data:\narrHumanDay1_2pm:", arrHumanDay1_2pm.shape, "  labelHumanDay1_2pm:", labelHumanDay1_2pm.shape, 
      "\narrHumanDay1_5pm:", arrHumanDay1_5pm.shape, "  labelHumanDay1_5pm:", labelHumanDay1_5pm.shape, "\narrHumanDay1_9pm:", 
      arrHumanDay1_9pm.shape, "  labelHumanDay1_9pm:", labelHumanDay1_9pm.shape, "\narrHumanDay1_3pm:", arrHumanDay1_3pm.shape,
      "  labelHumanDay1_3pm:", labelHumanDay1_3pm.shape, "\narrHumanDay1_6pm:", arrHumanDay1_6pm.shape, "  labelHumanDay1_6pm:", labelHumanDay1_6pm.shape)

print("\ncarV Data:\narrCarVday1_6:", arrCarVday1_6.shape, "  labelCarVday1_6:", labelCarVday1_6.shape,
      "\narrCarVday1_10:", arrCarVday1_10.shape, "  labelCarVday1_10:", labelCarVday1_10.shape, "\narrCarVday1_16:", 
      arrCarVday1_16.shape, "  labelCarVday1_16:", labelCarVday1_16.shape, "\narrCarVday1_19:", arrCarVday1_19.shape, 
      "  labelCarVday1_19:", labelCarVday1_19.shape, "\narrCarVday2_5:", arrCarVday2_5.shape, "  labelCarVday2_5:", labelCarVday2_5.shape, "\narrCarVday2_15:", 
      arrCarVday2_15.shape, "  labelCarVday2_15:", labelCarVday2_15.shape, "\narrCarVday2_11:", arrCarVday2_11.shape, 
      "  labelCarVday2_11:", labelCarVday2_11.shape, "\narrCarVday2_20:", arrCarVday2_20.shape, "  labelCarVday2_20:", labelCarVday2_20.shape)

print("\ncarH Data:\narrCarHday1_5:", arrCarHday1_5.shape, "  labelCarHday1_5:", labelCarHday1_5.shape, "\narrCarHday1_7:", 
      arrCarHday1_7.shape, "  labelCarHday1_7:", labelCarHday1_7.shape, "\narrCarHday1_11:", arrCarHday1_11.shape, 
      "  labelCarHday1_11:", labelCarHday1_11.shape, "\narrCarHday1_17:", arrCarHday1_17.shape, "  labelCarHday1_17:", labelCarHday1_17.shape, "\narrCarHday1_20:", 
      arrCarHday1_20.shape, "  labelCarHday1_20:", labelCarHday1_20.shape, "\narrCarHday2_2:", arrCarHday2_2.shape, 
      "  labelCarHday2_2:", labelCarHday2_2.shape, "\narrCarHday2_6:", arrCarHday2_6.shape, "  labelCarHday2_6:", labelCarHday2_6.shape, "\narrCarHday2_4:", 
      arrCarHday2_4.shape, "  labelCarHday2_4:", labelCarHday2_4.shape)

Human Data:
arrHumanDay1_2pm: (894, 128, 64)   labelHumanDay1_2pm: (894, 128, 64) 
arrHumanDay1_5pm: (998, 128, 64)   labelHumanDay1_5pm: (998, 128, 64) 
arrHumanDay1_9pm: (998, 128, 64)   labelHumanDay1_9pm: (998, 128, 64) 
arrHumanDay1_3pm: (997, 128, 64)   labelHumanDay1_3pm: (997, 128, 64) 
arrHumanDay1_6pm: (997, 128, 64)   labelHumanDay1_6pm: (997, 128, 64)

carV Data:
arrCarVday1_6: (345, 128, 64)   labelCarVday1_6: (345, 128, 64) 
arrCarVday1_10: (334, 128, 64)   labelCarVday1_10: (334, 128, 64) 
arrCarVday1_16: (383, 128, 64)   labelCarVday1_16: (383, 128, 64) 
arrCarVday1_19: (331, 128, 64)   labelCarVday1_19: (331, 128, 64) 
arrCarVday2_5: (998, 128, 64)   labelCarVday2_5: (998, 128, 64) 
arrCarVday2_15: (1001, 128, 64)   labelCarVday2_15: (1001, 128, 64) 
arrCarVday2_11: (994, 128, 64)   labelCarVday2_11: (994, 128, 64) 
arrCarVday2_20: (1000, 128, 64)   labelCarVday2_20: (1000, 128, 64)

carH Data:
arrCarHday1_5: (366, 128, 64)   labelCarHday1_5: (366, 128, 64) 
arrCarHday

In [24]:
inputArrays = np.concatenate((arrHumanDay1_2pm, arrCarVday1_6, arrCarHday1_5, arrCarVday1_10, arrCarHday1_7, 
                             arrHumanDay1_5pm, arrCarVday1_16, arrCarHday1_11, arrCarVday1_19, arrCarHday1_20, 
                             arrHumanDay1_9pm, arrCarVday2_5, arrCarHday1_17, arrCarVday2_15, arrCarHday2_2, 
                             arrHumanDay1_3pm, arrCarVday2_11, arrCarHday2_6, arrHumanDay1_6pm, arrCarVday2_20, 
                             arrCarHday2_4), axis=0)

targetArrays = np.concatenate((labelHumanDay1_2pm, labelCarVday1_6, labelCarHday1_5, labelCarVday1_10, labelCarHday1_7, 
                             labelHumanDay1_5pm, labelCarVday1_16, labelCarHday1_11, labelCarVday1_19, labelCarHday1_20, 
                             labelHumanDay1_9pm, labelCarVday2_5, labelCarHday1_17, labelCarVday2_15, labelCarHday2_2, 
                             labelHumanDay1_3pm, labelCarVday2_11, labelCarHday2_6, labelHumanDay1_6pm, labelCarVday2_20, 
                             labelCarHday2_4), axis=0)

inputArrays = np.reshape(inputArrays, (inputArrays.shape[0], 128, 64, 1))
targetArrays = np.reshape(targetArrays, (targetArrays.shape[0], 128, 64, 1))

inputArrays, targetArrays = shuffle(inputArrays, targetArrays, random_state=0)

In [25]:
# define the discriminator model

def define_discriminator(image_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # target image input
    in_target_image = Input(shape=image_shape)
    # concatenate images channel-wise
    merged = Concatenate()([in_src_image, in_target_image])
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [26]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add upsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # add batch normalization
    g = BatchNormalization()(g, training=True)
    # conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
    # merge with skip connection
    g = Concatenate()([g, skip_in])
    # relu activation
    g = Activation('relu')(g)
    return g

# define the standalone generator model
def define_generator(image_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    #e6 = define_encoder_block(e5, 512)
    #e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e5)
    b = Activation('relu')(b)
    # decoder model
    #d1 = decoder_block(b, e7, 512)
    #d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(b, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    # output
    g = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)
    return model

In [27]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model(in_src, [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [28]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, 4, 1))     # changed patch_shape argument to 4 
    return [X1, X2], y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
    # generate fake instance
    X = g_model.predict(samples)
    # create 'fake' class labels (0)
    y = zeros((len(X), patch_shape, 4, 1))    # changed patch_shape argument to 4
    return X, y

In [29]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realB[i])
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [30]:
# train pix2pix models
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)

In [31]:
# load image data
dataset = [inputArrays, targetArrays]

# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]

# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)

# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)

In [34]:
# display arrays/images in dataset

# load the dataset
src_images, tar_images = generate_fake_samples(g_model,inputArrays,4)
print('Loaded: ', src_images.shape, tar_images.shape)

# plot source images
n_samples = 3

pyplot.figure(figsize=(50, 20))

for i in range(n_samples):
    pyplot.subplot(2, n_samples, 10 + i)
    pyplot.axis('off')
    pyplot.imshow(src_images[i].astype('uint8'))
    
# plot target image
for i in range(n_samples):
    pyplot.subplot(2, n_samples,5 + n_samples + i)
    pyplot.axis('off')
    pyplot.imshow(tar_images[i].astype('uint8'))
    
pyplot.show()

Loaded:  (14855, 128, 64, 1) (14855, 4, 4, 1)


ValueError: ignored

<Figure size 3600x1440 with 0 Axes>

In [13]:
# train model
#train(d_model, g_model, gan_model, dataset)